In [1]:
import os
import pandas as pd
import re
import csv

chunks_folder = "chunks_2"  # Your chunks folder
chunks_by_pmid = {}

# Step 1: Load all chunks and group them by PMID
for csv_file in os.listdir(chunks_folder):
    if csv_file.endswith(".csv"):
        pmid = csv_file.split('_')[0]
        csv_path = os.path.join(chunks_folder, csv_file)
        df = pd.read_csv(csv_path)
        df['PMID'] = pmid
        if pmid not in chunks_by_pmid:
            chunks_by_pmid[pmid] = []
        chunks_by_pmid[pmid].append(df)

# Step 2: Concatenate all DataFrames per PMID into one DataFrame
chunks_by_pmid = {pmid: pd.concat(dfs, ignore_index=True) for pmid, dfs in chunks_by_pmid.items()}

# Step 3: Create batches of 2 PDFs
pmid_list = list(chunks_by_pmid.keys())
batch_size = 2

batches = [pmid_list[i:i + batch_size] for i in range(0, len(pmid_list), batch_size)]

top_n = 10  # or however many top chunks you want to retrieve per batch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# Initialize accelerator and get device
accelerator = Accelerator()
device = accelerator.device
model_name = "mistralai/Mistral-7B-v0.1"
token = "your_token"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token,
    cache_dir="/data/gent/490/vsc49096/huggingface"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    cache_dir="/data/gent/490/vsc49096/huggingface",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Ensure pad token is set correctly
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # or any other token you want to use as pad_token

model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
#pip uninstall peft -y
#pip install git+https://github.com/huggingface/peft.git
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer

# Load the model on CPU to avoid CUDA memory errors
model_st = SentenceTransformer('intfloat/multilingual-e5-large-instruct', device='cpu')

# Function to encode text on CPU
def embed_text(text, model):
    embeddings = model.encode(text, convert_to_tensor=True, device='cpu')
    return embeddings

query_texts = [
    
    "From the context below, extract all protein-protein interactions in the format: 'Protein A -> [interaction type] -> Protein B'. Only include clear interactions from the main body of the document.",
    "Identify any protein interactions mentioned and format them as: 'Protein A -> [interaction] -> Protein B'.",
    "List all protein-protein relations in the following form: 'Protein A -> [binding/modulation/activation/etc.] -> Protein B'.",
    "From the given context, return protein interaction pairs with interaction type in this format: 'Protein A -> [action] -> Protein B'.",
    "Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'."
]



for prompt_index, query_text in enumerate(query_texts, start=1):
    print(f"\n🧪 Testing Prompt {prompt_index}: {query_text}")

    query_embedding = embed_text(query_text, model_st)
    query_embedding_np = query_embedding.cpu().numpy().reshape(1, -1)

    ppi_results = []  # clear previous results

    for batch_num, pmid_batch in enumerate(batches, start=1):
        print(f"\n🔄 Processing batch {batch_num} with PMIDs: {pmid_batch}")

        batch_df = pd.concat([chunks_by_pmid[pmid] for pmid in pmid_batch], ignore_index=True)

        batch_embeddings = np.vstack(
            batch_df['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=',')).values
        )

        cosine_similarities = cosine_similarity(query_embedding_np, batch_embeddings)

        top_indices = cosine_similarities[0].argsort()[-top_n:][::-1]
        top_chunks = batch_df.iloc[top_indices].copy()
        top_chunks['cosine_similarity'] = cosine_similarities[0][top_indices]

        print(f"Top {top_n} relevant chunks from batch {batch_num}:")
        print(top_chunks[['page_number', 'sentence_chunk', 'cosine_similarity', 'PMID']])

        for i, row in top_chunks.iterrows():
            context = row["sentence_chunk"]
            pmid = row["PMID"]

            prompt = f"Question: {query_text}\nContext: {context}\nAnswer:"

            inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

            with torch.no_grad():
                output = model.generate(
                    **inputs,
                    max_new_tokens=200,
                    do_sample=False,
                    top_k=50,
                    temperature=0.7
                )

            answer = tokenizer.decode(output[0], skip_special_tokens=True)

            print(f"\n📄 PMID: {pmid}, Page: {row['page_number']}")
            print(f"📝 Prompt: {prompt}")
            print(f"🧠 Answer: {answer}")

            matches = re.findall(r'([\w\-]+)\s*->\s*\[?([\w\s\-]+)\]?\s*->\s*([\w\-]+)', answer)
            for match in matches:
                protein_1, interaction_type, protein_2 = match
                ppi_results.append({
                    "PMID": pmid,
                    "protein_1": protein_1,
                    "interaction_type": interaction_type.strip(),
                    "protein_2": protein_2
                })

    # Save CSV after all batches are processed for this prompt
    output_file = f"ppi_predictions_prompt_{prompt_index}.csv"
    os.makedirs("ppi_outputs", exist_ok=True)
    output_path = os.path.join("ppi_outputs", output_file)

    with open(output_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["PMID", "protein_1", "interaction_type", "protein_2"])
        writer.writeheader()
        writer.writerows(ppi_results)

    print(f"✅ Saved results for Prompt {prompt_index} to {output_path}")


🧪 Testing Prompt 1: From the context below, extract all protein-protein interactions in the format: 'Protein A -> [interaction type] -> Protein B'. Only include clear interactions from the main body of the document.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🔄 Processing batch 1 with PMIDs: ['7609053']
Top 10 relevant chunks from batch 1:
    page_number                                     sentence_chunk  \
18            5  A search of the Transcription Factor Database ...   
14            4  4A, lanes 1 and 2). This complex was signiﬁcan...   
19            5  These ﬁndings would not be incompatible with a...   
52            8  Xiao, J. H., I. Davidson, D. Ferrandon, R. Ros...   
12            3  In HL-60 cells, the wild-type HIV-2/CAT constr...   
5             2  71). Grifﬁn et al. have shown that activation ...   
41            8  46. Lu, Y., N. Touzjian, M. Stenzel, T. Dorfma...   
15            4  EMSAs using peripheral T-cell nuclear extracts...   
21            6  However, the means by which the peri-kB factor...   
17            5  in addition to these cis-acting elements there...   

    cosine_similarity     PMID  
18           0.824289  7609053  
14           0.814269  7609053  
19           0.807310  7609053  
52           0

/user/gent/490/vsc49096/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 5
📝 Prompt: Question: From the context below, extract all protein-protein interactions in the format: 'Protein A -> [interaction type] -> Protein B'. Only include clear interactions from the main body of the document.
Context: A search of the Transcription Factor Database (27) for similari- ties between the sequence of the peri-kB site and sequences of known transcription factor binding sites revealed some simi- larity to two known binding sites. One of these sites is located in the simian virus 40 enhancer and is known as the GT-IIB motif (ACAGCTG; 72), and the other is present in a number of cellular enhancers and binds helix-loop-helix proteins, and is known as the E2-box sequence (RCAGNTG; 54). It is not yet known whether the factors which bind these elements are sim- ilar to the peri-kB factor. A potential interaction of the peri-kB factor with NF-kB in mediating transcriptional activation is suggested by several ob- servations. The two enhancer sites lie i

...


📄 PMID: 7609053, Page: 4
📝 Prompt: Question: Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'.
Context: EMSAs using peripheral T-cell nuclear extracts showed two speciﬁc complexes which were signiﬁcantly diminished by an unlabeled peri-kB oligonucleotide but not by kB (Fig. 5B). Of note, the pattern of DNA-protein binding was markedly dif- ferent between the two cell types, suggesting the possibility of binding by different cell-speciﬁc proteins or by a protein(s) which is differentially modiﬁed in the two cell types, thus al- tering the binding pattern. In addition, when peripheral blood monocytes were activated with PMA and nuclear extracts from these cells used in EMSAs, the pattern of DNA-protein bind- ing was dramatically different from that seen in extracts from resting monocytes (Fig. 5C), consistent with the EMSA data from monocytic cell lines, suggesting that both constitutive and inducible 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 7
📝 Prompt: Question: Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'.
Context: Folks, T. M., J. Justement, A. Kinter, C. A. Dinarello, and A. S. Fauci. 1987. Cytokine-induced expression of HIV-1 in a chronically infected promono- cyte cell line. Science 238:800–802. 22. Galas, D. J., and A. Schmitz. 1978. DNase footprinting: a simple method for the detection of protein-DNA binding speciﬁcity. Nucleic Acids Res. 5:3157– 3170. 23. Gao, F., L. Yue, A. T. White, P. G. Pappas, J. Barchue, A. P. Hanson, B. M. Greene, P. M. Sharp, G. M. Shaw, and B. H. Hahn. 1992. Human infection by genetically diverse SIVSM-related HIV-2 in West Africa. Nature (Lon- don) 358:495–499.
Answer:
🧠 Answer: Question: Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'.
Context: Folks, T. M., J. Justement, A. Kinter, C. A. 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 7
📝 Prompt: Question: Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'.
Context: Mol. Cell. Biol. 2:1044–1051. 30. Grifﬁn, G. E., K. Leung, T. M. Folks, S. Kunkel, and G. J. Nabel. 1989. Activation of HIV gene expression during monocyte differentiation by in- duction of NF-kB. Nature (London) 339:662–669. 31. Gutierrez, C., R. R. Bernabe, J. Vega, and M. Kreisler. 1979. Puriﬁcation of human T and B cells by a discontinuous density gradient of PercollR. J. Immunol. Methods 29:57–63. 32. Guyader, M., M. Emerman, P. Soniga, F. Clavel, L. Montagnier, and M. Alizon. 1987.
Answer:
🧠 Answer: Question: Extract and format any explicit protein-protein interactions found using this template: 'Protein A -> [interaction type] -> Protein B'.
Context: Mol. Cell. Biol. 2:1044–1051. 30. Grifﬁn, G. E., K. Leung, T. M. Folks, S. Kunkel, and G. J. Nabel. 1989. Activation of HIV gene expression dur